In [1]:
%pip install git+https://github.com/isadoranun/FATS@aa81ce7f68d137798126b6e425dd069c3b4fde3a

  Cloning https://github.com/isadoranun/FATS (to revision aa81ce7f68d137798126b6e425dd069c3b4fde3a) to c:\users\lilu\appdata\local\temp\pip-req-build-htehup10
  Resolved https://github.com/isadoranun/FATS to commit aa81ce7f68d137798126b6e425dd069c3b4fde3a
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\lilu\appdata\local\programs\python\python312\lib\site-packages\fats-1.3.6-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
  Running command git clone --filter=blob:none --quiet https://github.com/isadoranun/FATS 'C:\Users\Lilu\AppData\Local\Temp\pip-req-build-htehup10'
  Running command git rev-parse -q --verify 'sha^aa81ce7f68d137798126b6e425dd069c3b4fde3a'
  Running command git fetch -q https://github.com/isadoranun/FATS aa81ce7f68d137798126b6e425dd069c3b4fde3a
  Running command git checkout -q aa81ce7f68d137798126b6e425dd069c3b4fde3a


In [ ]:
import csv
import io
import os
from warnings import filterwarnings

import boto3
import FATS
import numpy as np
import pandas as pd
from pyasassn.client import SkyPatrolClient
from sklearn.utils import resample
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

c:\Users\Lilu\AppData\Local\Programs\Python\Python312\Lib\site-packages\dask\dataframe\__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [2]:
filterwarnings('ignore', category=RuntimeWarning)

Загрузка данных

In [17]:
! aws configure
s3 = boto3.client('s3', endpoint_url='https://storage.yandexcloud.net')
bucket_name = 'guard-01'
response= s3.list_objects_v2(Bucket=bucket_name, Delimiter='/')

output_csv = "filenames.csv"
with open(output_csv, "w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    folders = [prefix['Prefix'] for prefix in response.get('CommonPrefixes', [])]
    for folder in folders[2:]:
        paginator = s3.get_paginator('list_objects_v2')
        pages = paginator.paginate(Bucket=bucket_name, Prefix=folder)
        objects = []

        n_objects = 0
        for i, page in enumerate(pages):
            if 'Contents' in page:
                if i % 50 == 0:
                    print(f"Page {i}. {n_objects} items processed.")
                n_objects += len(page['Contents'])
                for obj in page['Contents']:
                    if obj['Key'].endswith(".csv"):
                        writer.writerow(obj['Key'].split('/')[1:])

AWS Access Key ID [None]: YCAJEwlvD7seR1kGiUHgc2XX3
AWS Secret Access Key [None]: YCNMNFBWv2qBOkFJgoCsF-54lcmi00XI6PtjRMmG
Default region name [None]: ru-cental1-a
Default output format [None]: 
Page 0. 0 items processed.
Page 50. 50000 items processed.
Page 100. 100000 items processed.
Page 150. 150000 items processed.
Page 200. 200000 items processed.
Page 250. 250000 items processed.
Page 300. 300000 items processed.
Page 350. 350000 items processed.
Page 400. 400000 items processed.
Page 450. 450000 items processed.
Page 500. 500000 items processed.
Page 550. 550000 items processed.
Page 600. 600000 items processed.
Page 650. 650000 items processed.
Page 700. 700000 items processed.
Page 750. 750000 items processed.
Page 800. 800000 items processed.
Page 850. 850000 items processed.
Page 900. 900000 items processed.
Page 950. 950000 items processed.
Page 1000. 1000000 items processed.
Page 1050. 1050000 items processed.
Page 1100. 1100000 items processed.
Page 1150. 1150000 items p

In [ ]:
df = pd.read_csv('data\\filenames.csv', header=None)
df.columns = ['main_class', 'asas_sn_id']
df['asas_sn_id'] = df['asas_sn_id'].apply(lambda x: x.split('.')[0])
df = df[['asas_sn_id', 'main_class']]
df

,asas_sn_id,main_class
0,1002395,ECLIPSING
1,1003202,ECLIPSING
2,1003959,ECLIPSING
3,1004036,ECLIPSING
4,100590,ECLIPSING
...,...,...
1494795,661428625363,xray
1494796,661428693462,xray
1494797,661428895304,xray
1494798,8590170053,xray


In [6]:
df['main_class'].value_counts()

main_class
ECLIPSING      578603
PULSATING      453156
ROTATING       434944
ERUPTIVE        13477
cataclysmic      8582
OTHERS           6000
xray               38
Name: count, dtype: int64

In [ ]:
client = SkyPatrolClient()
client.catalogs.aavsovsx

Welcome to ASAS-SN Skypatrol!

Current Deployment Version: 0.6.17 (26 JAN 2024)
Please upgrade your client if not up to date.



,col_names,dtypes
0,asas_sn_id,bigint
1,ra_deg,double
2,dec_deg,double
3,source_number,bigint
4,name,string
5,variability_flag,bigint
6,lii,double
7,bii,double
8,variability_type,string
9,max_mag_type,double


In [ ]:
Var_dict = {
    'ECLIPSING':['E','EA','EB','E-DO','EP','EW','EC','ED','ESD','AR','BD','D','DM','DS','DW','EL','GS','HW','K','KE','KW','PN','SD','WD'],
    'ROTATING': ['ACV','BY','CTTS','ROT','INT','IT','ELL','FKCOM','HB','LERI','PSR','R','ROT','RS','SXARI','SXARI','E','TTS','ROT','WTTS','ROT','NSIN', 'ELL','PSR','ROT','RS'],
    'PULSATING':['ACEP', 'ACEP(B)', 'ACEPS', 'ACYG', 'BCEP', 'BCEPS', 'BLAP', 'BXCIR', 'CEP', 'CW', 'CWA', 'CWB', 'CWB(B)', 'CWBS', 'DCEP', 'DCEP(B)', 'CEP(B)', 'DCEPS', 'DCEPS(B)', 'DSCT', 'DSCTC', 'DWLYN', 'GDOR', 'HADS', 'HADS(B)', 'L', 'LB', 'LC', 'M', 'ORG', 'PPN', 'PVTEL', 'PVTELI', 'PVTELII', 'PVTELIII', 'roAm', 'roAp', 'ACVO', 'RR', 'RRAB', 'RRC', 'RRD', 'RR(B)', 'RV', 'RVA', 'RVB', 'SPB', 'LPB', 'SPBe', 'SR', 'SRA', 'SRB', 'SRC', 'SRD', 'SRS', 'SXPHE', 'SXPHE(B)', 'V361HYA', 'RPHS', 'V1093HER', 'ZZ', 'ZZA', 'ZZB', 'ZZ/GWLIB', 'ZZO', 'ZZLep', 'LPV', 'CW-FO', 'CW-FU', 'DCEP-FO', 'DCEP-FU', 'DSCTr', 'PULS', '(B)', 'BL', 'GWLIB', 'O'],
    'ERUPTIVE':['BE', 'cPNB[e]', 'CTTS', 'DPV', 'DYPer', 'EXOR', 'FF', 'FSCMa', 'FUOR', 'FU', 'GCAS', 'I', 'IA', 'IB', 'IN', 'INA', 'INAT', 'INB', 'INS', 'INSA', 'INSB', 'INST', 'INT', 'IS', 'ISA', 'ISB', 'RCB', 'SDOR', 'TTS', 'UV', 'UVN', 'UXOR', 'WR', 'WTTS', 'ZZA/O', 'YSO', 'DIP', 'WR', '(YY)'],
    'CATACLYSMIC':['AM', 'CBSS', 'CBSS/V', 'DQ', 'DQ/AE', 'IBWD', 'N', 'NA', 'NB', 'NC', 'NL', 'NL/VY', 'NR', 'SN', 'SN I', 'SN Ia', 'SN Iax', 'SN Ia-00cx-like', 'SN Ia-02es-like', 'SN Ia-06gz-like', 'SN Ia-86G-like', 'SN Ia-91bg-like', 'SN Ia-91T-like', 'SN Ia-99aa-like', 'SN Ia-Ca-rich', 'SN Ia-CSM', 'SN Ib', 'SN Ic', 'SN Icn', 'SN Ic-BL', 'SN Idn', 'SN Ien', 'SN II', 'SN IIa', 'SN IIb', 'SN IId', 'SN II-L', 'SN IIn', 'SN II-P', 'SN-pec', 'SLSN', 'SLSN-I', 'SLSN-II', 'UG', 'UGER', 'UGSS', 'UGSU', 'UGWZ', 'UGZ', 'UGZ/IW', 'V838MON', 'WDP', 'ZAND', 'CV', 'IBWD', 'V', 'VY'],
    'XRAY':['HMXB', 'IMXB', 'LMXB', 'X', 'BHXB', 'XB', 'XJ', 'XN', 'XP', 'XPR', 'XBR'],
    'others':['L:', 'S', '*'],
}

for key, value in Var_dict.items():
    value = tuple(value)
    query = f"""
    SELECT
    *
    FROM aavsovsx
    WHERE variability_type IN {value} and variability_flag=0
    ORDER BY asas_sn_id
    """
    df_name = f"df_{key.lower()}"
    globals()[df_name] = client.adql_query(query)[['asas_sn_id', 'variability_type']]
    print(f"Создана переменная: {df_name}")


Создана переменная: df_eclipsing
Создана переменная: df_rotating
Создана переменная: df_pulsating
Создана переменная: df_eruptive
Создана переменная: df_cataclysmic
Создана переменная: df_xray
Создана переменная: df_others


In [9]:
conc = pd.concat([df_eclipsing, df_rotating, df_pulsating, df_eruptive, df_cataclysmic,
           df_xray, df_others], axis=0)
conc.drop_duplicates(inplace=True)
conc

,asas_sn_id,variability_type
0,421,EW
1,2445,E
2,2546,EW
3,4618,EW
4,4998,EW
...,...,...
13585,661428965476,L:
13586,661428965524,L:
13587,661428965534,L:
13588,661428965590,L:


In [10]:
df['asas_sn_id'] = df['asas_sn_id'].astype('int64')
df.dtypes

asas_sn_id     int64
main_class    object
dtype: object

In [11]:
conc.duplicated().sum()

0

In [12]:
df_new = pd.merge(df, conc, on='asas_sn_id', how='inner')
df_new

,asas_sn_id,main_class,variability_type
0,1002395,ECLIPSING,EW
1,1003202,ECLIPSING,EW
2,1003959,ECLIPSING,EW
3,1004036,ECLIPSING,EW
4,100590,ECLIPSING,EA
...,...,...,...
1494798,661428625363,xray,HMXB
1494799,661428693462,xray,X
1494800,661428895304,xray,HMXB
1494801,8590170053,xray,HMXB


In [13]:
df_new['main_class'].value_counts()

main_class
ECLIPSING      578604
PULSATING      453157
ROTATING       434945
ERUPTIVE        13477
cataclysmic      8582
OTHERS           6000
xray               38
Name: count, dtype: int64

In [258]:
df_new.to_csv('id_mainclass_varclss.csv', index=False)

In [15]:
df_new.isna().sum()

asas_sn_id          0
main_class          0
variability_type    0
dtype: int64

In [16]:
classes_for_model = {"ECLIPSING": ["E", "EW", "EA", "EB"],
                    "PULSATING": ["SR", "RRAB", "SRS", "RRC", "L", "M",
                                "DSCT", "SRB", "HADS", "LB", "DCEP"],
                    "ROTATING": ["ROT", "ELL", "RS", "BY", "ACV"],
                    "ERUPTIVE": ["UV", "TTS", "GCAS"],
                    "cataclysmic": ["UG", "UGSU"]
                    }

In [17]:
# Создаем маску для фильтрации
mask = df_new.apply(
    lambda row: (
        row["main_class"] in classes_for_model  # Проверяем наличие класса в словаре
        and row["variability_type"] in classes_for_model[row["main_class"]]  # Проверяем тип
    ),
    axis=1
)

# Применяем маску
filtered_df = df_new[mask]

In [250]:
filtered_df

,asas_sn_id,main_class,variability_type
0,1002395,ECLIPSING,EW
1,1003202,ECLIPSING,EW
2,1003959,ECLIPSING,EW
3,1004036,ECLIPSING,EW
4,100590,ECLIPSING,EA
...,...,...,...
1494754,94489523523,cataclysmic,UGSU
1494756,94489954000,cataclysmic,UGSU
1494758,94490107581,cataclysmic,UG
1494759,94490226906,cataclysmic,UG


In [19]:
filtered_df['main_class'].value_counts()

main_class
ECLIPSING      482169
PULSATING      435981
ROTATING        94703
cataclysmic      6323
ERUPTIVE         3842
Name: count, dtype: int64

In [259]:
filtered_df.to_csv('id_mainclass_varclss_filtered.csv', index=False)

Балансировка - UNDERsampling всех подклассов

In [31]:
# Фильтрация данных

balance_strategies = {
            "ECLIPSING": {
                "under": {"E": 10000, "EW": 10000, "EA": 6000, "EB": 2000},
                "remain": {},
                "over": {}
            },
            "PULSATING": {
                "under": {"SR": 10000, "RRAB": 10000, "SRS": 10000, "RRC": 5000,
                          "L": 5000, "M": 4000, "DSCT": 2000, "SRB": 1000,
                          "HADS": 1000, "LB": 1000, "DCEP": 1000},
                "remain": {},
                "over": {}
            },
            "ROTATING": {
                "under": {"ROT": 6000, "ELL": 4000},
                "remain": {"RS": 2000, "BY": 1000, "ACV": 1000},
                "over": {}
            },
            "ERUPTIVE": {
                "under": {},
                "remain": {"UV": 1000, "TTS": 1000},
                "over": {"GCAS": 1000}
            },
            "cataclysmic": {
                "under": {},
                "remain": {"UG": 2000},
                "over": {"UGSU": 1000}
            }
            }

balanced_ids = []

for main_class, strategies in balance_strategies.items():
    class_data = filtered_df[filtered_df["main_class"] == main_class]

    for strategy, subcl in strategies.items():
        if strategy == 'under':
            for sub, target in subcl.items():
                subclass_data = class_data[class_data["variability_type"] == sub]
                subclass_data = resample(subclass_data, n_samples=target, random_state=42, replace=False)
                balanced_ids.extend(subclass_data["asas_sn_id"].tolist())
        else:
            for sub, target in subcl.items():
                subclass_data = class_data[class_data["variability_type"] == sub]
                balanced_ids.extend(subclass_data["asas_sn_id"].tolist())

balanced_labels = filtered_df[filtered_df["asas_sn_id"].isin(balanced_ids)]

In [32]:
# до
filtered_df.groupby(['main_class','variability_type']).count()

asas_sn_id
main_class  variability_type            
ECLIPSING   E                     338364
            EA                     49312
            EB                     11260
            EW                     83233
ERUPTIVE    GCAS                     942
            TTS                     1055
            UV                      1845
PULSATING   DCEP                    2277
            DSCT                    5972
            HADS                    2585
            L                      33486
            LB                      2470
            M                      20380
            RRAB                   90026
            RRC                    34751
            SR                    166584
            SRB                     3540
            SRS                    73910
ROTATING    ACV                     1397
            BY                      3393
            ELL                    25436
            ROT                    59419
            RS                      5058
cataclysmic UG                      5523
            UGSU                     800

In [47]:
# после
balanced_labels.groupby(['main_class','variability_type']).count()

asas_sn_id
main_class  variability_type            
ECLIPSING   E                      10000
            EA                      6000
            EB                      2000
            EW                     10000
ERUPTIVE    GCAS                     942
            TTS                     1055
            UV                      1845
PULSATING   DCEP                    1000
            DSCT                    2000
            HADS                    1000
            L                       5000
            LB                      1000
            M                       4000
            RRAB                   10000
            RRC                     5000
            SR                     10000
            SRB                     1000
            SRS                    10000
ROTATING    ACV                     1397
            BY                      3393
            ELL                     4000
            ROT                     6000
            RS                      5058
cataclysmic UG                      5523
            UGSU                     800

In [48]:
balanced_labels.groupby(['main_class']).count()

,asas_sn_id,variability_type
main_class,,
ECLIPSING,28000,28000
ERUPTIVE,3842,3842
PULSATING,50000,50000
ROTATING,19848,19848
cataclysmic,6323,6323


In [ ]:
balanced_labels.to_csv('data\\balanced_labels_100thsd.csv', index=False)

Объединенная подготовка для извлечения FATS признаков и подготовка данных для CNN

In [ ]:
class S3TimeSeriesProcessor:
    """Processes astronomical time series data from S3 storage for different ML tasks.

    Handles CSV files containing light curve data, supporting:
    - Feature extraction for traditional ML
    - Data preparation for CNN models
    - Combined processing modes

    Args:
        bucket_name (str): Name of S3 bucket containing source data
        aws_access_key_id (str): AWS-compatible access key for S3 auth
        aws_secret_access_key (str): AWS-compatible secret key for S3 auth

    Attributes:
        s3 (boto3.client): Configured S3 client with Yandex Cloud endpoint
        bucket (str): Target bucket name for all operations
        scaler (StandardScaler): Scaler instance for data normalization

    Methods:
        process_prefix: Main processing method for S3 prefixes
        jd_to_mjd: Static method for Julian Day conversion
    """
    
    
    def __init__(self, bucket_name, aws_access_key_id, aws_secret_access_key):
        """Initializes S3 client and processing components."""
        self.s3 = boto3.client('s3',
                             aws_access_key_id=aws_access_key_id,
                             aws_secret_access_key=aws_secret_access_key,
                             endpoint_url='https://storage.yandexcloud.net')
        self.bucket = bucket_name
        self.scaler = StandardScaler()


    @staticmethod
    def jd_to_mjd(jd):
        """Converts Julian Day to Modified Julian Day (MJD).

        Args:
            jd (float): Julian Day value

        Returns:
            float: MJD value. Returns NaN for invalid inputs (negative/NaN JD).

        Formula:
            MJD = JD - 2,400,000.5
        """
        if np.isnan(jd) or jd < 0:
            return np.nan
        return jd - 2400000.5


    def _process_file(self, file_key, processing_type, max_length=100):
        """Internal method for processing individual S3 files.

        Args:
            file_key (str): Full S3 object key/path
            processing_type (str): Processing mode - 'features', 'cnn', or 'both'
            max_length (int): Sequence length for CNN padding (default: 100)

        Returns:
            varies: Based on processing_type:
                - 'features': pandas.DataFrame with extracted features
                - 'cnn': numpy.ndarray of shaped (max_length, 2)
                - 'both': dict with 'features' and 'cnn_data' keys
                - None for invalid files/processing errors

        Note:
            - Performs common preprocessing: NaN removal, JD->MJD conversion
            - Requires CSV format with header row and columns: mag, jd, mag_err
            - Skips files with <10 valid observations after preprocessing
        """
        try:
            # Загрузка и базовый препроцессинг
            response = self.s3.get_object(Bucket=self.bucket, Key=file_key)
            content = response['Body'].read().decode('utf-8')
            
            df = pd.read_csv(io.StringIO(content), header=1)
            
            # Принудительное преобразование типов
            required_cols = {'mag', 'jd', 'mag_err'}
            for col in required_cols:
                df[col] = pd.to_numeric(df[col], errors='coerce')

            # Общая предобработка
            df = df[['mag', 'jd', 'mag_err']].dropna(how='any', axis=0).\
                                              drop_duplicates(subset='jd')

            mask = (
                np.isfinite(df['mag']) &
                np.isfinite(df['jd']) &
                np.isfinite(df['mag_err'])
            )
            df = df[mask].copy()
            if df.empty:
                return None, None

            df.loc[:, 'jd'] = df.loc[:, 'jd'].apply(self.jd_to_mjd)
            df = df.dropna(subset=['jd'], how='any').sort_values('jd')
            
            # Удаление выбросов
            mean_mag_err = df['mag_err'].mean()
            mask_err = (df['mag_err'] < 3 * mean_mag_err)

            mean_mag = df['mag'].mean()
            std_mag = df['mag'].std()
            if std_mag == 0:
                mask_mag = pd.Series(True, index=df.index)
            else:
                mask_mag = ((np.abs(df['mag'] - mean_mag) / std_mag) < 5)

            df = df[mask_err & mask_mag].copy()

            # Если слишком мало данных в файле
            if len(df) < 100:
                return None, None

            # Разделение обработки
            if processing_type == 'features':
                return self._extract_features(df, file_key)

            elif processing_type == 'cnn':
                return self._prepare_cnn_data(df, max_length)

            elif processing_type == 'both':
                features = self._extract_features(df, file_key, return_df=True)
                cnn_data = self._prepare_cnn_data(df, max_length)
                return {**(features if features is not None else {}), 'cnn_data': cnn_data}

        except Exception as e:
            print(f"Error processing {file_key}: {str(e)}")
            return None, None


    def _extract_features(self, df, file_key, return_df=False):
        """Internal feature extraction using FATS library.

        Args:
            df (pd.DataFrame): Preprocessed light curve data
            file_key (str): Source file path for ID extraction
            return_df (bool): Whether to return raw data (default: False)

        Returns:
            pd.DataFrame/None: Features DataFrame, or None on failure
            dict/None: If return_df=True, adds raw data arrays under 'raw_data'

        Note:
            Computes 17 statistical features using FATS library
            Object ID is extracted from filename (basename without extension)
        """
        try:
            mag = df['mag'].values
            time = df['jd'].values
            error = df['mag_err'].values

            #preprocessed_data = FATS.Preprocess_LC(mag, time, error)
            #mag, time, error = preprocessed_data.Preprocess()

            features = FATS.FeatureSpace(
                featureList=['Amplitude','Mean','Std','Skew','MedianAbsDev'
                                ,'PeriodLS','Period_fit','Freq1_harmonics_amplitude_0',
                                'FluxPercentileRatioMid20','FluxPercentileRatioMid80',
                                'PercentAmplitude','LinearTrend','MaxSlope',
                                'SlottedA_length','StructureFunction_index_21',
                                'Autocor_length', 'StetsonK'],
                Data=['magnitude', 'time', 'error']
            ).calculateFeature(np.vstack([mag, time, error])).result(method='dict')

            file_id = file_key.split('/')[-1].split('.')[0]
            features_df = pd.DataFrame([features]).assign(object_id=file_id)

            return features_df if not return_df else {
                'features': features_df,
                'raw_data': (mag, time, error)
            }

        except Exception as e:
            #print(f"Feature extraction failed for {file_key}: {str(e)}")
            return None


    def _prepare_cnn_data(self, df, max_length):
        """Internal method for CNN input preparation.

        Args:
            df (pd.DataFrame): Preprocessed light curve data
            max_length (int): Target sequence length

        Returns:
            numpy.ndarray: Shaped (max_length, 2) array containing:
                - StandardScaler-normalized [mag, mag_err] values
                - Zero-padded if shorter than max_length
                - Truncated if longer than max_length
        """
        try:
            scaled = self.scaler.fit_transform(df[['mag', 'mag_err']])

            if len(scaled) > max_length:
                padded = scaled[:max_length]
            else:
                padded = np.pad(scaled,
                            ((0, max_length - len(scaled)), (0, 0)),
                            mode='constant',
                            constant_values=0)
                
            # Проверка окончательной формы
            if padded.shape != (max_length, 2):
                print(f"Invalid shape after padding: {padded.shape}")
                return None
                
            return padded
        except Exception as e:
            print(f"Error in _prepare_cnn_data: {str(e)}")
            return None


    def process_prefix(self, prefix, ids_list, processing_type='both', max_length=100):
        """Processes all files in S3 prefix matching object IDs.

        Args:
            prefix (str): S3 prefix (folder path) to process
            ids_list (list[int]): List of numeric object IDs to include
            processing_type (str): One of ['features', 'cnn', 'both'] (default: 'both')
            max_length (int): Padding length for CNN data (default: 100)

        Returns:
            tuple: (features_df, cnn_data) where:
                - features_df: pandas.DataFrame of extracted features (None if empty)
                - cnn_data: numpy.ndarray of shaped (n_samples, max_length, 2) (None if empty)

        Note:
            - Saves results to S3 automatically:
                - Features: {prefix}/features.csv
                - CNN data: {prefix}/cnn_data.npz
            - Uses pagination to handle large prefix contents
            - Processes ~50 files before printing progress update
            - Skips files with invalid formats/non-numeric names
        """
        features_df = pd.DataFrame()
        cnn_data = []
        ids = []
        i=0

        try:
            paginator = self.s3.get_paginator('list_objects_v2')
            page_iterator = paginator.paginate(Bucket=self.bucket, Prefix=prefix)

            for page in page_iterator:
                if 'Contents' not in page:
                    print(f"Префикс '{prefix}' не содержит объектов. Пропускаю...")
                    continue
                
                for obj in page['Contents']:
                    file_key = obj['Key']
                    if not file_key.endswith('.csv'):
                        continue

                    try:
                      file_id = int(file_key.split('/')[-1].split('.')[0])
                    except ValueError:
                      print(f"Skipping invalid file: {file_key}")
                      continue
                    if file_id not in ids_list:
                      continue

                    result = self._process_file(file_key, processing_type, max_length)
                    i+=1
                    if i % 100 == 0:
                      print(f"{i} items processed.")

                    if result is None:
                        continue

                    if processing_type == 'features':
                        features_df = pd.concat([features_df, result], ignore_index=True)
                    elif processing_type == 'cnn':
                        if isinstance(result, np.ndarray) and result.shape == (max_length, 2):
                            cnn_data.append(result)
                            ids.append(str(file_id))
                    elif processing_type == 'both':
                        if 'features' not in result or 'cnn_data' not in result:
                            continue
                        if result['cnn_data'].shape == (max_length, 2):
                            features_df = pd.concat([features_df, result['features']], ignore_index=True)
                            cnn_data.append(result['cnn_data'])
                            ids.append(str(file_id))


            # Сохранение результатов
            if processing_type in ['features', 'both'] and not features_df.empty:
                csv_buffer = io.StringIO()
                features_df.to_csv(csv_buffer, index=False)
                self.s3.put_object(
                    Bucket=self.bucket,
                    Key=f"{prefix}/features_100thsnd.csv",
                    Body=csv_buffer.getvalue()
                )

            if processing_type in ['cnn', 'both'] and cnn_data:
                with io.BytesIO() as buffer:
                    np.savez(buffer,
                            X=np.array(cnn_data),
                            ids=np.array(ids))
                    buffer.seek(0)
                    self.s3.put_object(
                        Bucket=self.bucket,
                        Key=f"{prefix}/cnn_data_100thsnd.npz",
                        Body=buffer.getvalue()
                    )
                    
            print(f'Результат {prefix} сохранен')
            
        except Exception as e:
            print(f"Ошибка при обработке префикса {prefix, file_key}: {str(e)}")

        features_result = features_df if not features_df.empty else None
        cnn_result = np.array(cnn_data) if cnn_data else None
        return (features_result, cnn_result)



# Варианты обработки:
#processor = S3TimeSeriesProcessor('your-bucket-name', aws_access_key_id, aws_secret_access_key)
# 1. Только фичи
#features_df, _ = processor.process_prefix('raw_data/', ids, processing_type='features')
# 2. Только данные для CNN
#_, cnn_data = processor.process_prefix('raw_data/', ids, processing_type='cnn')
# 3. Оба варианта одновременно
#full_features, full_cnn = processor.process_prefix('raw_data/', ids, processing_type='both')

In [ ]:
# Настройки S3
aws_access_key_id = os.getenv('aws_access_key_id')
aws_secret_access_key = os.getenv('aws_secret_access_key')
S3_BUCKET = 'guard-01'

In [ ]:
# Основной процесс выполнения
prefixs = ['variables/ECLIPSING/', 'variables/ERUPTIVE/',
           'variables/PULSATING/', 'variables/ROTATING/',
          'variables/cataclysmic/']
          

processor = S3TimeSeriesProcessor(S3_BUCKET,
                                  aws_access_key_id,
                                  aws_secret_access_key)

for pref in prefixs:
    ids_label = pref.split('/')[1]
    ids = balanced_labels[balanced_labels['main_class'] == ids_label]\
                                        ['asas_sn_id'].values.tolist()
    
    df_name = f"df_{pref.replace('/', '_')}"
    try:
      features, cnn = processor.process_prefix(pref,
                                                ids_list=ids,
                                                processing_type='cnn',
                                                max_length=500)

      if features is not None and not features.empty:
        print(f"Обработано объектов: {len(features)}")

      if cnn is not None:
        print(f"CNN данных: {cnn.shape}")
    except Exception as e:
        print(f"Ошибка при обработке папки {pref}: {str(e)}")

Подготовка данных из S3 Object Storage для модели

In [103]:
categories = ['cataclysmic', 'ECLIPSING', 'ERUPTIVE', 
              'PULSATING', 'ROTATING']

In [104]:
# Загрузка обработанных данных после выполнения process_prefix
# Загрузка features-файлов
for category in categories:
    file_path = f'data\\{category}features_100thsnd.csv'
    df_name = f'{category}_features_df'
    globals()[df_name] = pd.read_csv(file_path)

In [116]:
df_names = [f"{cat}_features_df" for cat in categories]
combined_features_df = pd.concat([globals()[name] for name in df_names], \
            axis=0, ignore_index=True)

In [117]:
combined_features_df.head(5)

,Amplitude,Mean,Std,Skew,MedianAbsDev,PeriodLS,Period_fit,Freq1_harmonics_amplitude_0,FluxPercentileRatioMid20,FluxPercentileRatioMid80,PercentAmplitude,LinearTrend,MaxSlope,SlottedA_length,StructureFunction_index_21,Autocor_length,StetsonK,object_id
0,2.453048,17.065320,1.138409,-2.687498,0.275713,140.004952,4.553269e-13,0.449467,0.048963,0.319600,0.291868,-0.000038,605.912236,10.012418,0.978943,4.0,0.613548,103079638041
1,1.329324,17.543929,0.707306,-0.760725,0.425654,29.553694,1.683327e-09,0.276514,0.152484,0.794296,0.126842,0.000081,225.040294,6.065751,1.608388,2.0,0.585476,103079712924
2,1.949295,17.244249,1.118498,-1.498699,0.360145,1.018004,8.863602e-07,0.357907,0.079785,0.824872,0.205719,-0.000066,633.376221,6.112814,2.185448,2.0,0.695065,103079869726
3,0.437794,15.366972,0.225751,0.943857,0.138493,2425.897004,2.672834e-09,0.095487,0.179524,0.759858,0.063058,0.000024,399.352675,2.518532,2.062003,1.0,0.788289,103080663269
4,1.195628,17.332857,0.699284,-0.208403,0.571028,29.544342,1.968770e-17,0.368001,0.229901,0.867717,0.135238,0.000080,685.866114,4.363533,1.763024,2.0,0.471525,111670140260


In [118]:
combined_features_df.shape

(95358, 18)

In [119]:
combined_features_df.isna().sum()

Amplitude                      0
Mean                           0
Std                            0
Skew                           0
MedianAbsDev                   0
PeriodLS                       0
Period_fit                     0
Freq1_harmonics_amplitude_0    0
FluxPercentileRatioMid20       0
FluxPercentileRatioMid80       0
PercentAmplitude               0
LinearTrend                    0
MaxSlope                       0
SlottedA_length                0
StructureFunction_index_21     0
Autocor_length                 0
StetsonK                       0
object_id                      0
dtype: int64

In [120]:
combined_features_df.duplicated().sum()

0

In [110]:
# Загрузка NPZ-файлов
for category in categories:
    file_path = f'data\\{category}cnn_data_100thsnd.npz'
    npz_data = np.load(file_path)
    globals()[f'{category}_ts_data'] = npz_data

In [111]:
# Инициализируем пустые списки для накопления данных
X_list = []
ids_list = []
for category in categories:
    X = globals()[f'{category}_ts_data']['X']
    ids = globals()[f'{category}_ts_data']['ids']
    X_list.append(X)
    ids_list.append(ids)
    
# Объединяем все данные
X_ts_combined = np.concatenate(X_list, axis=0)  # shape: (Total_samples, 100, 2)
ids_ts_combined = np.concatenate(ids_list, axis=0)  # shape: (Total_samples,)

print(X_ts_combined.shape)
print(ids_ts_combined.shape)

(108011, 500, 2)
(108011,)


In [112]:
X_ts_combined[:,:,0].shape

(108011, 500)

In [150]:
ids_ts_combined = ids_ts_combined.astype('int64')

In [151]:
# Находим пересечение ID между features и временными рядами
common_ids = np.intersect1d(combined_features_df['object_id'], ids_ts_combined)
common_ids.shape

(95358,)

In [152]:
# Фильтруем features
combined_features_df = combined_features_df[combined_features_df['object_id'].isin(common_ids)]
# Фильтруем временные ряды
ts_mask = np.isin(ids_ts_combined, common_ids)
X_ts_combined = X_ts_combined[ts_mask]
ids_ts_combined = ids_ts_combined[ts_mask]

# Сортируем оба набора по ID для гарантии соответствия
combined_features_df = combined_features_df.sort_values('object_id').reset_index(drop=True)
sort_idx = np.argsort(ids_ts_combined)
X_ts_combined = X_ts_combined[sort_idx]
ids_ts_combined = ids_ts_combined[sort_idx]

# Проверка совпадения
assert np.array_equal(combined_features_df['object_id'].values, ids_ts_combined), \
    f"Mismatch in {category} after filtering"

# Проверка размерностей
print("Filtered shapes:")
print("X_ts_combined:", X_ts_combined.shape) 
print("Features:", combined_features_df.shape)

Filtered shapes:
X_ts_combined: (95358, 500, 2)
Features: (95358, 18)


In [155]:
# Создаем случайное перемешивание по индексам
np.random.seed(42)
n_samples = len(combined_features_df)
shuffled_indices = np.random.permutation(n_samples)

combined_features_df = combined_features_df.iloc[shuffled_indices].reset_index(drop=True)
X_ts_combined = X_ts_combined[shuffled_indices]
ids_ts_combined = ids_ts_combined[shuffled_indices]

# Проверка результата
print("Первые 5 ID после перемешивания:")
print("DataFrame IDs:", combined_features_df['object_id'].head().values)
print("NPZ IDs:", ids_ts_combined[:5])

Первые 5 ID после перемешивания:
DataFrame IDs: [661427982892 661427566257 566936322939 283468103486 661427843236]
NPZ IDs: [661427982892 661427566257 566936322939 283468103486 661427843236]


In [156]:
# Добавим по индексам основные классы и подклассы
combined_features_df['object_id']

0        661427982892
1        661427566257
2        566936322939
3        283468103486
4        661427843236
             ...     
95353    103080615449
95354    661427982398
95355    661428443816
95356     17179995839
95357    257698605443
Name: object_id, Length: 95358, dtype: int64

In [157]:
balanced_labels = pd.read_csv('data\\balanced_labels_100thsd.csv')
balanced_labels

,asas_sn_id,main_class,variability_type
0,1007291,ECLIPSING,EB
1,1023864,ECLIPSING,EA
2,1026073,ECLIPSING,EA
3,103079216708,ECLIPSING,EB
4,103079241652,ECLIPSING,EB
...,...,...,...
108008,94489523523,cataclysmic,UGSU
108009,94489954000,cataclysmic,UGSU
108010,94490107581,cataclysmic,UG
108011,94490226906,cataclysmic,UG


In [158]:
combined_features_tempdf = pd.merge(combined_features_df, balanced_labels, 
                                    left_on='object_id', right_on='asas_sn_id', 
                                    how='left').drop(columns=['object_id'])
combined_features_tempdf.head(5)

,Amplitude,Mean,Std,Skew,MedianAbsDev,PeriodLS,Period_fit,Freq1_harmonics_amplitude_0,FluxPercentileRatioMid20,FluxPercentileRatioMid80,PercentAmplitude,LinearTrend,MaxSlope,SlottedA_length,StructureFunction_index_21,Autocor_length,StetsonK,asas_sn_id,main_class,variability_type
0,0.505012,15.355483,0.286298,-0.683286,0.121730,4731.971722,1.709626e-87,0.340246,0.089873,0.891953,0.062943,2.113399e-04,591.605895,0.167989,1.508197,105.0,0.781274,661427982892,ROTATING,RS
1,0.281494,15.925608,0.142973,0.011181,0.094343,1.000073,2.653599e-06,0.048720,0.173953,0.757959,0.041931,-4.636019e-07,13347.850155,0.041070,2.099814,1.0,0.816171,661427566257,ECLIPSING,EA
2,0.313620,14.448348,0.209633,-1.181723,0.037838,22994.870902,3.388257e-208,2.010872,0.042873,0.923184,0.043869,1.578735e-04,69.258233,0.183217,1.302553,220.0,0.775535,566936322939,ROTATING,ROT
3,0.224287,11.342643,0.115876,1.078561,0.058107,3112.690134,1.618432e-87,0.099671,0.105703,0.786368,0.077900,5.451522e-05,824122.098070,0.000007,1.124551,38.0,0.815059,283468103486,PULSATING,RRC
4,3.906747,16.748555,2.436347,-1.778324,0.440184,3781.999751,1.416309e-100,2.661332,0.057606,0.939809,0.498681,1.763291e-03,1089.826014,0.079933,1.399981,112.0,0.801041,661427843236,PULSATING,SRS


In [159]:
y_labels = combined_features_tempdf[['asas_sn_id', 
                                     'main_class', 
                                     'variability_type']]

In [160]:
# Сохраним полученные объединенные признаки и целевые переменные
combined_features_df.to_csv('data\\shuffled_combined_100thsnd.csv', index=False)
print(f"CSV сохранен: data\\shuffled_combined_100thsnd.csv ({len(combined_features_df)} строк)")

y_labels.to_csv('data\\y_labels_100thsnd.csv', index=False)
print(f"CSV сохранен: data\\y_labels_100thsnd.csv ({len(y_labels)} строк)")

# Сохранение перемешанных NPZ
np.savez(
    'data\\shuffled_combined_100thsnd.npz',
    X=X_ts_combined,
    ids=ids_ts_combined
)
print("NPZ сохранен: data\\shuffled_combined_100thsnd.npz")

CSV сохранен: data\shuffled_combined_100thsnd.csv (95358 строк)
CSV сохранен: data\y_labels_100thsnd.csv (95358 строк)
NPZ сохранен: data\shuffled_combined_100thsnd.npz
